In [1]:
# | default_exp 01-intro-tutorial-by-jeremy/part4

In [2]:
# | export
from fasthtml.common import *

In [3]:
from fasthtml.jupyter import JupyUvi, HTMX
from fasthtml_tutorials.core import start_server, stop_server, show_with_pico

### Basic Todo Application

In [4]:
# This is to render the pico component with styling in the output cells
# if your using VS code with dark theme then change it to light colored theme to properly see the outputs
show(picocondlink)

In [5]:
# To properly render the pico component, we need to add the "pico" class to every tag that is displayed in the output, and this is what this function does
set_pico_cls()

<IPython.core.display.Javascript object>

In [6]:
app, rt = fast_app(live=True)
start_server(app)

In [7]:
# In the tutorial jeremy changes the database file to `data/todos.db` in the end, but for best practices we are including it from the start.
app, rt, todos, Todo = fast_app(
    "data/todos.db", live=True, id=int, title=str, done=bool, pk="id"
)
start_server(app)

In [8]:
@rt("/")
def get():
    items = todos()
    return Titled(
        "Todos",
        Div(*items),
    )

In [9]:
@rt("/")
def get():
    todos.insert(Todo(title="First todo", done=False))
    items = todos()
    return Titled(
        "Todos",
        Div(*items),
    )

In [10]:
# To add a new todo, we need to refresh the page
HTMX(link=True)

In [11]:
items = todos()
show(
    Titled(
        "Todos",
        Div(*items),
    )
)

In [12]:
items = [Li(o) for o in todos()]
show(Ul(*items))

In [13]:
@rt("/")
def get():
    todos.insert(Todo(title="Second todo", done=False))
    items = [Li(o) for o in todos()]
    return Titled(
        "Todos",
        Ul(*items),
    )

In [14]:
HTMX(link=True)

In [15]:
def render(todo):
    return Li(todo.title)


app, rt, todos, Todo = fast_app(
    "data/todos.db",
    live=True,
    render=render,
    id=int,
    title=str,
    done=bool,
    pk="id",
)
start_server(app)

In [16]:
show(Ul(*todos()))

In [17]:
@rt("/")
def get():
    return Titled(
        "Todos",
        Ul(*todos()),
    )

In [18]:
def render(todo):
    return Li(todo.title + ("✅ " if todo.done else ""))


app, rt, todos, Todo = fast_app(
    "data/todos.db",
    live=True,
    render=render,
    id=int,
    title=str,
    done=bool,
    pk="id",
)
start_server(app)

Updating the todos

In [19]:
def render(todo):
    tid = f"todo-{todo.id}"
    toggle = A("Toggle", hx_get=f"/toggle/{todo.id}", target_id=tid)
    return Li(
        toggle,
        todo.title + ("✅ " if todo.done else ""),
        id=tid,
    )


app, rt, todos, Todo = fast_app(
    "data/todos.db",
    live=True,
    render=render,
    id=int,
    title=str,
    done=bool,
    pk="id",
)
start_server(app)

In [20]:
Ul(*todos())

```html
<ul></ul>

```

In [21]:
show(Ul(*todos()))

In [22]:
@rt("/")
def get():
    return Titled(
        "Todos",
        Ul(*todos()),
    )

In [23]:
@rt("/toggle/{tid}")
def get(tid: int):
    todo = todos[tid]
    todo.done = not todo.done
    return todos.update(todo)

Delete a todo

In [24]:
# | export
def render(todo):
    tid = f"todo-{todo.id}"
    toggle = A("Toggle", hx_get=f"/toggle/{todo.id}", target_id=tid)
    delete = A("Delete", hx_delete=f"/{todo.id}", target_id=tid, hx_swap="outerHTML")
    return Li(
        toggle,
        delete,
        todo.title + ("✅ " if todo.done else ""),
        id=tid,
    )


app, rt, todos, Todo = fast_app(
    "data/todos.db",
    live=True,
    render=render,
    id=int,
    title=str,
    done=bool,
    pk="id",
)

In [25]:
start_server(app)

In [26]:
Ul(*todos())

```html
<ul></ul>

```

In [27]:
show(Ul(*todos()))

In [28]:
@rt("/")
def get():
    return Titled(
        "Todos",
        Ul(*todos()),
    )


@rt("/toggle/{tid}")
def get(tid: int):
    todo = todos[tid]
    todo.done = not todo.done
    return todos.update(todo)

In [29]:
@rt("/{tid}")
def delete(tid: int):
    todos.delete(tid)

Create Todo

In [30]:
# TODO: The output is not displayed as expected, why?
# seems like set_pico_cls() function is not working.

show(Group(Input(placeholder="Add a new todo", name="title"), Button("Add")))

In [31]:
# Thus we are using custom funtion to apply the styling
# this function is nothing but a wrapper around the elements with the class 'pico'.
show_with_pico(Group(Input(placeholder="Add a new todo", name="title"), Button("Add")))

In [32]:
frm = Form(
    Group(Input(placeholder="Add a new todo", name="title"), Button("Add")),
    hx_post="/",
    target_id="todo-list",
    hx_swap="beforeend",
)

show_with_pico(
    Card(
        Ul(*todos()),
        header=frm,
    )
)

In [33]:
@rt("/")
def get():
    frm = Form(
        Group(Input(placeholder="Add a new todo", name="title"), Button("Add")),
        hx_post="/",
        target_id="todo-list",
        hx_swap="beforeend",
    )
    return Titled(
        "Todos",
        Card(
            Ul(*todos(), id="todo-list"),
            header=frm,
        ),
    )

In [34]:
@rt("/")
def post(todo: Todo):
    return todos.insert(todo)

Changing two object at the same time

In [35]:
# | export
def mk_input():
    return Input(placeholder="Add a new todo", id="title", hx_swap_oob="true")

In [36]:
# | export
@rt("/")
def get():
    frm = Form(
        Group(mk_input(), Button("Add")),
        hx_post="/",
        target_id="todo-list",
        hx_swap="beforeend",
    )
    return Titled(
        "Todos",
        Card(
            Ul(*todos(), id="todo-list"),
            header=frm,
        ),
    )

In [37]:
# | export
@rt("/")
def post(todo: Todo):
    return todos.insert(todo), mk_input()

In [38]:
# | export
@rt("/{tid}")
def delete(tid: int):
    todos.delete(tid)


@rt("/toggle/{tid}")
def get(tid: int):
    todo = todos[tid]
    todo.done = not todo.done
    return todos.update(todo)

In [39]:
HTMX(link=True)

In [40]:
stop_server()

In [41]:
# | export
serve()

In [42]:
import nbdev

nbdev.nbdev_export()